# Implementing the Gradient Descent Algorithm + Interactive Visualizer

In this lab, we'll implement the basic functions of the Gradient Descent algorithm to find the boundary in a small dataset. Moreover, an interactive Visualizer will be implemented in order to see the the effect on the Cross-Enropy of each iteration. First, we'll start with some functions that will help us plot and visualize the data.

Please

In [28]:
import numpy as np
import pandas as pd
from plotly.offline import plot
import plotly.graph_objects as go
import plotly.io as pio


# Plotting function for both points and lines
def plot_routine(fig, data, errors, m, b):
    
    data['errors'] = errors
     
    x1 = np.arange(-1, 3, 1) 
    x2 = m*x1+b            
    
    data = [go.Scatter(mode='markers', visible=False, marker_color=data["colors"], marker=dict(size=data['errors'], 
                        sizeref = 0.025, line=dict(color='Black', width=1)), x=data["x1"], y=data["x2"],
                        hoverinfo="text", hovertemplate= "Cross-Entropy: %{marker.size:.3f}" + "<extra></extra>"),
            go.Scatter(mode="lines",visible=False, x = x1, y = x2, line=dict(color="black"))]
    
    fig.add_traces(data)

# Plotting function for the slider
def plot_slider(fig):
    
    steps= []
    
    # Define the steps dict, for each step of the slider we want just two dataset to be visible:
    # one boundary_line and the related set of points (with the related error)
    for i in range(int(len(fig.data)/2)):
        step = dict(method="restyle",args=["visible", [False] * len(fig.data)])
        
        step["args"][1][2*i] = True  # Toggle i'th trace to "visible"
        step["args"][1][2*i+1] = True  # Toggle i'th trace to "visible"
        steps.append(step)
    
    active_level = 0    # Active Level of dataset when the plot is generated
    fig.data[active_level].visible = True
    fig.data[active_level+1].visible = True
    
    sliders = [dict(
    active= active_level,   # Slider starting level
    currentvalue={"prefix": "Iteration: "},
    pad={"t": 50}, 
    steps=steps)]
    
    # Updating the plot with the designed slider
    fig.update_layout(sliders=sliders)



## Basic functions

- Sigmoid activation function

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

- Output (prediction) formula

$$\hat{y} = \sigma(w_1 x_1 + w_2 x_2 + b)$$

- Error function

$$Error(y, \hat{y}) = - y \log(\hat{y}) - (1-y) \log(1-\hat{y})$$

- The function that updates the weights

$$ w_i \longrightarrow w_i + \alpha (y - \hat{y}) x_i$$

$$ b \longrightarrow b + \alpha (y - \hat{y})$$

In [29]:
# Activation (sigmoid) function, Continuous function 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Prediction function based on the sigmoid function (continuos output)
def output_formula(features, weights, bias):
    return sigmoid(np.dot(features, weights) + bias)

# Error function based on the Cross-Entropy for a 2 dimension set
def error_formula(y, output):
    return - y*np.log(output) - (1 - y) * np.log(1-output)

# Perceptron weights/bias update, based on the Gradient Descent
def update_weights(x, y, weights, bias, learnrate):
    output = output_formula(x, weights, bias)
    d_error = y - output
    weights += learnrate * d_error * x
    bias += learnrate * d_error
    return weights, bias

## Training function
This function will help us iterate the gradient descent algorithm through all the data, for a number of epochs. It will also plot the data, and some of the boundary lines obtained as we run the algorithm.

In [30]:
# Training Algorithm
def train(data, features, targets, epochs, learnrate, graph_lines=False):
    
    # Plot Init, layout settings
    layout = go.Layout(
    showlegend=False,
    title = "Gradient Descent Algorithm",
    template = pio.templates['plotly'],
    xaxis = dict(range=[-0.01, 1.01]),
    yaxis = dict(range=[-0.01, 1.01]))
    
    fig = go.Figure(layout=layout)
    
    #Arrays Init
    errors = []
    n_records, n_features = features.shape
    last_loss = None
    weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
    bias = 0
    
    # Iterations
    for e in range(epochs):
        
        # Weights and Bias update based on the entire data set 
        for x, y in zip(features, targets):
            weights, bias = update_weights(x, y, weights, bias, learnrate)
        
        # Prediction function (outuput continuos value == propability to be classify right)
        out = output_formula(features, weights, bias)
        
        # log-loss error on the entire training set (Continuos Error)
        loss = np.mean(error_formula(targets, out))
        errors.append(loss)
        
        # Printing Train log-loss error data
        if e % (epochs / 10) == 0:
            print("\n========== Epoch", e,"==========")
            if last_loss and last_loss < loss:
                #If loss increase (every epochs/100 steps) we have an alert 
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
            
            # Accuracy on the entire training set (Discrete Error)
            predictions = out > 0.5
            accuracy = np.mean(predictions == targets)
            print("Accuracy: ", accuracy)
        
        # Plot Boundary lines and points + Error as Size every epochs/100 steps
        if graph_lines and e % (epochs / 100) == 0:
            
            output = output_formula(features, weights, bias)       
            point_errors = error_formula(targets,output)
            
            plot_routine(fig, data, point_errors, -weights[0]/weights[1], -bias/weights[1])
    
    # Slider implementation
    plot_slider(fig)
    
    #Plotting
    plot(fig)

## Reading  and Preparing the data

In [31]:
# Data extraction
data = pd.read_csv('DataSet1.csv', header=None)
X = np.array(data[[0,1]])
y = np.array(data[2])

data.columns = ["x1","x2","y"]

# Associate '0' and '1' with 'red' and 'blue'
colors = []
for i in range(len(data['y'])):
    if data['y'][i] == 1:
       colors.append('red')
    else:
       colors.append('blue') 

data['colors'] = colors

## Training the Algorithm!

In the graph that is outputed, we have a slider that gives the posibility to see each Iteration of the Algorithm. 
Moving the slider right, we can se how the boundary line moves closer to the ideal position. 

The size of each points decrease as the Cross-Entropy (now on CE) decrease. Remember that the CE represent the inverse of the propability of a point to be classified right. Bigger the CE, lower the probability that the point has been classify correctly.  

Play with the paramters below (epochs and learnrate), to see how the result of the Algorithm changes.


In [33]:
# Setting the random seed, feel free to change it and see different solutions.
np.random.seed()

# Input Parameter, Epochs represent the number of updates/iterations, Learning Rate the step
# between consecutive updates 
epochs = 100   # MIN = 100
learnrate = 0.01

# Main Program execution    
train(data, X, y, epochs, learnrate, True)


========== Epoch 0 ==========
Train loss:  0.7927919394151404
Accuracy:  0.48

========== Epoch 10 ==========
Train loss:  0.6569602877606614
Accuracy:  0.5

========== Epoch 20 ==========
Train loss:  0.5820698219229617
Accuracy:  0.67

========== Epoch 30 ==========
Train loss:  0.5232551140606432
Accuracy:  0.8

========== Epoch 40 ==========
Train loss:  0.4768761340345054
Accuracy:  0.88

========== Epoch 50 ==========
Train loss:  0.4397318048929534
Accuracy:  0.92

========== Epoch 60 ==========
Train loss:  0.4094966649977223
Accuracy:  0.93

========== Epoch 70 ==========
Train loss:  0.384501319649033
Accuracy:  0.92

========== Epoch 80 ==========
Train loss:  0.36354165945669936
Accuracy:  0.93

========== Epoch 90 ==========
Train loss:  0.34573937809486516
Accuracy:  0.93
